In [1]:
using LinearAlgebra
using RowEchelon # Necesario para la función rref()

"""
    pseudoinversa_full_rank(A::AbstractMatrix)

Calcula la pseudoinversa de Moore-Penrose para una matriz A utilizando el
algoritmo de factorización de rango completo, como se describe en las imágenes.

El algoritmo sigue estos pasos:
a) Reducir A a su forma escalonada reducida por filas (E_A).
b) Formar la matriz B con las columnas pivote de A.
c) Formar la matriz C con las filas no nulas de E_A.
d) Calcular las inversas de (C*C') y (B'*B).
e) Aplicar la fórmula: A† = C' * inv(C*C') * inv(B'*B) * B'

# Argumentos
- `A::AbstractMatrix`: La matriz de entrada (real o compleja).

# Retorna
- `Matrix`: La pseudoinversa de Moore-Penrose (A†).
"""
function pseudoinversa_full_rank(A::AbstractMatrix)
    
    # --- a) Reducir A a su forma escalonada por filas E_A ---
    # Usamos la función rref() del paquete RowEchelon
    E_A = rref(A)
    println("a) Forma escalonada reducida por filas (E_A):\n", E_A, "\n")
    
    # --- b) Seleccionar las columnas distinguidas (pivote) para formar B ---
    # Identificamos las columnas pivote buscando el primer '1' en cada fila no nula de E_A
    pivot_cols = []
    # Usamos una pequeña tolerancia para comparar con cero
    tol = 1e-8 
    for i in 1:size(E_A, 1)
        # findfirst busca el índice del primer elemento que no es cero en la fila
        pivot_index = findfirst(x -> abs(x) > tol, E_A[i, :])
        if pivot_index !== nothing
            push!(pivot_cols, pivot_index)
        end
    end
    unique!(pivot_cols) # Nos aseguramos de que los índices son únicos
    sort!(pivot_cols)
    
    B = A[:, pivot_cols]
    println("b) Columnas pivote de A: ", pivot_cols)
    println("   Matriz B:\n", B, "\n")
    
    # --- c) Seleccionar las filas distintas de cero de E_A para formar C ---
    nonzero_rows = []
    for i in 1:size(E_A, 1)
        # any() verifica si algún elemento de la fila es distinto de cero
        if any(x -> abs(x) > tol, E_A[i, :])
            push!(nonzero_rows, i)
        end
    end
    
    C = E_A[nonzero_rows, :]
    println("c) Filas no nulas de E_A: ", nonzero_rows)
    println("   Matriz C:\n", C, "\n")
    
    # --- d) Calcular (C*C')⁻¹ y (B'*B)⁻¹ ---
    # En Julia, el apóstrofo (') es la transpuesta conjugada (adjunta)
    # que corresponde a la notación C* y B*
    B_star = B'
    C_star = C'
    
    inv_B_star_B = inv(B_star * B)
    inv_C_C_star = inv(C * C_star)
    
    println("d) Calculando inversas...")
    println("   (B'*B)⁻¹:\n", inv_B_star_B)
    println("   (C*C')⁻¹:\n", inv_C_C_star, "\n")
    
    # --- e) Calcular A† con la fórmula final ---
    # A† = C* (CC*)⁻¹ (B*B)⁻¹ B*
    A_pseudoinverse = C_star * inv_C_C_star * inv_B_star_B * B_star
    println("e) Calculando la pseudoinversa A† con la fórmula final.\n")
    
    return A_pseudoinverse
end

pseudoinversa_full_rank

In [2]:
# Matriz A del ejemplo proporcionado en las imágenes
A = [1.0 1.0 0.0 1.0;
     2.0 2.0 0.0 2.0;
     3.0 3.0 2.0 3.0;
     4.0 4.0 1.0 1.0]

println("--- Calculando la Pseudoinversa para la Matriz A ---")
println("Matriz Original (A):\n", A, "\n")

# Llamamos a nuestra función
A_pinv_calculada = pseudoinversa_full_rank(A)

println("--- RESULTADO FINAL ---")
println("Pseudoinversa calculada con el algoritmo A†:\n", round.(A_pinv_calculada, digits=5), "\n")


# --- VERIFICACIÓN ---
# Julia tiene una función incorporada para esto en LinearAlgebra
A_pinv_nativa = pinv(A)
println("--- VERIFICACIÓN CON LA FUNCIÓN pinv() DE JULIA ---")
println("Pseudoinversa calculada por pinv(A):\n", round.(A_pinv_nativa, digits=5), "\n")

# Comprobamos si los resultados son iguales (usando isapprox para floats)
if isapprox(A_pinv_calculada, A_pinv_nativa)
    println("✅ ¡Éxito! El resultado coincide con la función nativa de Julia.")
else
    println("❌ Error: El resultado NO coincide con la función nativa de Julia.")
end

--- Calculando la Pseudoinversa para la Matriz A ---
Matriz Original (A):
[1.0 1.0 0.0 1.0; 2.0 2.0 0.0 2.0; 3.0 3.0 2.0 3.0; 4.0 4.0 1.0 1.0]

a) Forma escalonada reducida por filas (E_A):
[1.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0]

b) Columnas pivote de A: Any[1, 3, 4]
   Matriz B:
[1.0 0.0 1.0; 2.0 0.0 2.0; 3.0 2.0 3.0; 4.0 1.0 1.0]

c) Filas no nulas de E_A: Any[1, 2, 3]
   Matriz C:
[1.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]

d) Calculando inversas...
   (B'*B)⁻¹:
[0.1444444444444444 -0.1333333333333333 -0.11111111111111108; -0.13333333333333328 0.6999999999999997 -0.16666666666666657; -0.11111111111111108 -0.16666666666666657 0.2777777777777777]
   (C*C')⁻¹:
[0.5 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]

e) Calculando la pseudoinversa A† con la fórmula final.

--- RESULTADO FINAL ---
Pseudoinversa calculada con el algoritmo A†:
[0.01667 0.03333 -0.08333 0.16667; 0.01667 0.03333 -0.08333 0.16667; -0.3 -0.6 0.5 0.0; 0.16667 0.33333 0.16667 -0.33333]

--